# MONAI vs YOLOv8 模型对比分析

本notebook用于对比MONAI和YOLOv8模型在脑肿瘤分割任务上的性能。

## 功能：
1. 加载两个模型的评估结果
2. 对比各项指标（Dice系数、Loss等）
3. 可视化对比结果
4. 生成对比报告


## 1. 安装依赖和挂载Google Drive


In [ ]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 导入必要的库
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

print("库导入完成")


## 2. 配置路径和加载结果


In [ ]:
# 模型结果路径
MONAI_RESULTS_PATH = "/content/drive/MyDrive/brain-tumor-models-monai/test_results.json"
YOLO_MODEL_PATH = "/content/drive/MyDrive/brain-tumor-models"  # YOLOv8模型保存路径

# 分割类别
SEGMENT_CLASSES = {
    0: 'Background',
    1: 'NECROTIC/CORE',
    2: 'EDEMA',
    3: 'ENHANCING'
}

print(f"MONAI结果路径: {MONAI_RESULTS_PATH}")
print(f"YOLOv8模型路径: {YOLO_MODEL_PATH}")


## 3. 加载MONAI模型结果


In [ ]:
# 加载MONAI模型结果
monai_results = None
if os.path.exists(MONAI_RESULTS_PATH):
    with open(MONAI_RESULTS_PATH, 'r') as f:
        monai_results = json.load(f)
    print("✅ MONAI结果加载成功")
    print(f"  测试Loss: {monai_results.get('test_loss', 'N/A')}")
    print(f"  平均Dice系数: {monai_results.get('test_dice_mean', 'N/A')}")
    print(f"  各类别Dice: {monai_results.get('test_dice_per_class', 'N/A')}")
else:
    print("⚠️ 未找到MONAI结果文件，请先运行训练notebook")
    print(f"  预期路径: {MONAI_RESULTS_PATH}")


## 4. 加载YOLOv8模型结果（如果存在）


In [ ]:
# 尝试加载YOLOv8模型结果
# 注意：YOLOv8的结果可能保存在不同的格式中，需要根据实际情况调整
yolo_results = None

# 方法1: 查找JSON格式的结果文件
yolo_json_paths = [
    os.path.join(YOLO_MODEL_PATH, "test_results.json"),
    os.path.join(YOLO_MODEL_PATH, "evaluation_results.json"),
    os.path.join(YOLO_MODEL_PATH, "results.json"),
]

for path in yolo_json_paths:
    if os.path.exists(path):
        with open(path, 'r') as f:
            yolo_results = json.load(f)
        print(f"✅ YOLOv8结果加载成功: {path}")
        break

if yolo_results is None:
    print("⚠️ 未找到YOLOv8结果文件")
    print("  提示：如果YOLOv8模型已训练，请手动评估并保存结果")
    print("  或者运行YOLOv8的评估代码，将结果保存为JSON格式")
    print(f"  搜索路径: {YOLO_MODEL_PATH}")
    
    # 显示可用的文件
    if os.path.exists(YOLO_MODEL_PATH):
        print(f"\n  YOLOv8目录下的文件:")
        files = os.listdir(YOLO_MODEL_PATH)
        for f in files[:10]:  # 只显示前10个
            print(f"    - {f}")


## 5. 对比分析（如果两个模型结果都存在）


In [ ]:
# 对比分析函数
def compare_models(monai_results, yolo_results):
    """对比两个模型的性能"""
    
    if monai_results is None:
        print("❌ MONAI结果不存在，无法对比")
        return None
    
    if yolo_results is None:
        print("⚠️ YOLOv8结果不存在，只显示MONAI结果")
        print("\n" + "="*60)
        print("MONAI模型结果:")
        print("="*60)
        print(f"测试Loss: {monai_results.get('test_loss', 'N/A'):.4f}")
        print(f"平均Dice系数: {monai_results.get('test_dice_mean', 'N/A'):.4f}")
        
        dice_per_class = monai_results.get('test_dice_per_class', [])
        if dice_per_class:
            print(f"\n各类别Dice系数:")
            class_names = [SEGMENT_CLASSES[i] for i in range(1, 4)]
            for idx, class_name in enumerate(class_names):
                if idx < len(dice_per_class):
                    print(f"  {class_name}: {dice_per_class[idx]:.4f}")
                else:
                    print(f"  {class_name}: N/A")
        
        hausdorff = monai_results.get('test_hausdorff_mean')
        if hausdorff is not None:
            print(f"平均Hausdorff距离: {hausdorff:.4f}")
        else:
            print(f"平均Hausdorff距离: 计算失败（跳过）")
        
        return None
    
    # 两个模型都存在，进行对比
    print("="*60)
    print("模型对比结果")
    print("="*60)
    
    # 创建对比表格
    comparison_data = {
        '指标': ['测试Loss', '平均Dice系数'],
        'MONAI': [
            f"{monai_results.get('test_loss', 0):.4f}",
            f"{monai_results.get('test_dice_mean', 0):.4f}"
        ],
        'YOLOv8': [
            f"{yolo_results.get('test_loss', 0):.4f}",
            f"{yolo_results.get('test_dice_mean', 0):.4f}"
        ]
    }
    
    # 添加Hausdorff距离（如果存在）
    monai_hd = monai_results.get('test_hausdorff_mean')
    yolo_hd = yolo_results.get('test_hausdorff_mean')
    if monai_hd is not None or yolo_hd is not None:
        comparison_data['指标'].append('平均Hausdorff距离')
        comparison_data['MONAI'].append(f"{monai_hd:.4f}" if monai_hd is not None else "N/A")
        comparison_data['YOLOv8'].append(f"{yolo_hd:.4f}" if yolo_hd is not None else "N/A")
    
    df = pd.DataFrame(comparison_data)
    print("\n整体性能对比:")
    print(df.to_string(index=False))
    
    # 各类别Dice系数对比
    print("\n" + "="*60)
    print("各类别Dice系数对比:")
    print("="*60)
    
    monai_dice = monai_results.get('test_dice_per_class', [])
    yolo_dice = yolo_results.get('test_dice_per_class', [])
    
    class_names = [SEGMENT_CLASSES[i] for i in range(1, 4)]
    class_comparison = []
    
    for idx, class_name in enumerate(class_names):
        monai_val = monai_dice[idx] if idx < len(monai_dice) else None
        yolo_val = yolo_dice[idx] if idx < len(yolo_dice) else None
        
        class_comparison.append({
            '类别': class_name,
            'MONAI': f"{monai_val:.4f}" if monai_val is not None else "N/A",
            'YOLOv8': f"{yolo_val:.4f}" if yolo_val is not None else "N/A"
        })
        
        # 判断哪个更好
        if monai_val is not None and yolo_val is not None:
            if monai_val > yolo_val:
                winner = "MONAI更好"
            elif yolo_val > monai_val:
                winner = "YOLOv8更好"
            else:
                winner = "相同"
            class_comparison[-1]['优势'] = winner
        else:
            class_comparison[-1]['优势'] = "无法比较"
    
    df_class = pd.DataFrame(class_comparison)
    print(df_class.to_string(index=False))
    
    return comparison_data, class_comparison

# 执行对比
comparison_result = compare_models(monai_results, yolo_results)


In [ ]:
# 可视化对比结果
if monai_results is not None:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # 1. 平均Dice系数对比
    if yolo_results is not None:
        models = ['MONAI', 'YOLOv8']
        dice_scores = [
            monai_results.get('test_dice_mean', 0),
            yolo_results.get('test_dice_mean', 0)
        ]
        
        axes[0].bar(models, dice_scores, color=['blue', 'orange'])
        axes[0].set_ylabel('平均Dice系数')
        axes[0].set_title('平均Dice系数对比')
        axes[0].set_ylim([0, 1])
        axes[0].grid(True, alpha=0.3)
        
        # 添加数值标签
        for i, v in enumerate(dice_scores):
            axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')
        
        # 2. 各类别Dice系数对比
        monai_dice = monai_results.get('test_dice_per_class', [])
        yolo_dice = yolo_results.get('test_dice_per_class', [])
        class_names = [SEGMENT_CLASSES[i] for i in range(1, 4)]
        
        x = np.arange(len(class_names))
        width = 0.35
        
        monai_vals = [monai_dice[i] if i < len(monai_dice) else 0 for i in range(len(class_names))]
        yolo_vals = [yolo_dice[i] if i < len(yolo_dice) else 0 for i in range(len(class_names))]
        
        axes[1].bar(x - width/2, monai_vals, width, label='MONAI', color='blue')
        axes[1].bar(x + width/2, yolo_vals, width, label='YOLOv8', color='orange')
        axes[1].set_xlabel('类别')
        axes[1].set_ylabel('Dice系数')
        axes[1].set_title('各类别Dice系数对比')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(class_names, rotation=45, ha='right')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        axes[1].set_ylim([0, 1])
    else:
        # 只显示MONAI结果
        dice_score = monai_results.get('test_dice_mean', 0)
        axes[0].bar(['MONAI'], [dice_score], color='blue')
        axes[0].set_ylabel('平均Dice系数')
        axes[0].set_title('MONAI模型 - 平均Dice系数')
        axes[0].set_ylim([0, 1])
        axes[0].grid(True, alpha=0.3)
        axes[0].text(0, dice_score + 0.01, f'{dice_score:.4f}', ha='center', va='bottom')
        
        # 各类别Dice系数
        monai_dice = monai_results.get('test_dice_per_class', [])
        class_names = [SEGMENT_CLASSES[i] for i in range(1, 4)]
        
        x = np.arange(len(class_names))
        monai_vals = [monai_dice[i] if i < len(monai_dice) else 0 for i in range(len(class_names))]
        
        axes[1].bar(x, monai_vals, color='blue')
        axes[1].set_xlabel('类别')
        axes[1].set_ylabel('Dice系数')
        axes[1].set_title('MONAI模型 - 各类别Dice系数')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(class_names, rotation=45, ha='right')
        axes[1].grid(True, alpha=0.3)
        axes[1].set_ylim([0, 1])
    
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/brain-tumor-models-monai/comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("对比图表已保存")
else:
    print("无法生成对比图表：MONAI结果不存在")


## 7. 总结和建议

### 当前状态分析：

**MONAI模型：**
- 当前只训练了7个epoch
- Dice系数: 0.0138（非常低）
- 说明：模型还未充分训练

**建议：**
1. **继续训练更多epoch**（至少50-100个epoch）
2. **检查训练曲线**，确认模型是否在收敛
3. **调整超参数**（学习率、batch size等）
4. **检查数据预处理**是否正确

### 何时进行对比：

✅ **可以进行初步对比**（即使结果不理想）：
- 了解两个模型的训练进度
- 检查代码是否正确运行
- 验证评估流程

⚠️ **需要充分训练后再对比**（获得准确结论）：
- 两个模型都训练到收敛
- Dice系数达到合理水平（通常>0.5）
- 在相同的测试集上评估

### 下一步：

1. 继续训练MONAI模型到更多epoch
2. 如果YOLOv8模型已训练，加载其评估结果
3. 在相同测试集上评估两个模型
4. 进行详细的性能对比
